In [1]:
import multiprocessing
multiprocessing.set_start_method('spawn')
#multiprocessing.set_start_method('fork')
#multiprocessing.set_start_method('forkserver')


In [2]:
from Electromagnetics.FDTD import *
import numpy as np
import time
import multiprocessing
import logging
import sys, os
multiprocessing.log_to_stderr(logging.ERROR)
#sys.stdout = open(str(os.getpid()) + ".out", "w")

tic = time.perf_counter()

n_dim = 2
r0, r1, dr = [None]*3
pr_grid = None
S = 0.9
n_t = None

omega = 1.0

s_pml = None
d_pml = None
set_pml = False

if n_dim==3:
    r0 = np.array([0.0, 0.0, 0.0])
    r1 = np.array([8.0, 1.0, 1.0])
    dr = np.array([0.02, 0.02, 0.02])
    S /= np.sqrt(3)
    pr_grid = np.array([8, 1, 1])
    n_t = 200
    
    d_pml = np.array([0.1, 0.1, 0.1])
    s_pml = np.array([1.0+1.0j, 1.0+1.0j, 1.0+1.0j])
    
elif n_dim==2:
    r0 = np.array([0.0, 0.0])
    r1 = np.array([10.0, 2.0])
    dr = np.array([0.01, 0.02])
    S /= np.sqrt(2)
    pr_grid = np.array([8, 1])
    n_t = 1000

    d_pml = np.array([0.1, 0.1])
    s_pml = np.array([1.0+1.0j, 1.0+1.0j])

elif n_dim==1:
    r0 = np.array([0.0])
    r1 = np.array([200.0])
    dr = np.array([0.01])
    pr_grid = np.array([8])
    n_t = 10000
    
    d_pml = np.array([0.1])
    s_pml = np.array([1.0+1.0j])
    
dt = S*min(dr)
d_v = np.prod(dr)

fdtd = FDTDMPYEE(r0, r1, dr, dt, pr_grid)

fdtd.SendStrMessage('self.AllocateSideArr_list(["E", "D", "Je", "eps"])')
fdtd.SendStrMessage('self.AllocateFaceArr_list(["B", "H", "Jm"])')
fdtd.SendStrMessage('self.SetSpatialPointsSide()')
fdtd.SendStrMessage('self.SetSpatialPointsFace()')

fdtd.SendStrMessage('self.SetupFdtdVars(FVTypes.E, None, var=self.E)')
fdtd.SendStrMessage('self.SetupFdtdVars(FVTypes.H, None, var=self.H)')
fdtd.SendStrMessage('self.SetupFdtdVars(FVTypes.D, None, var=self.D)')
fdtd.SendStrMessage('self.SetupFdtdVars(FVTypes.B, None, var=self.B)')

fdtd.getProcessPMLs(d_pml, s_pml)
print('fdtd.pml_grid:\n', fdtd.pml_grid)

r_c = (r0+r1)/2.0
    
r_j = r_c
r_j_str = '['+','.join(map(str, r_j.tolist()))+']' 
je_args = "{'r0':%s, 'mag':1.0, 'f_t':lambda t: np.sin(%s*t), 'src_dir':'z'}"%(r_j_str, omega)
fdtd.SendStrMessage("self.SetupFdtdVars(FVTypes.JePoint, %s, var=self.Je)"%(je_args))

w_eps = (r1-r0)/2
r0_eps = r_c - w_eps/2
r1_eps = r_c + w_eps/2
r0_eps_str = '['+','.join(map(str, r0_eps.tolist()))+']' 
r1_eps_str = '['+','.join(map(str, r1_eps.tolist()))+']' 
eps_args = "{'r0':%s, 'r1':%s, 'mag_in':4.0, 'mag_out':1.0}"%(r0_eps_str, r1_eps_str)
fdtd.SendStrMessage("self.SetupFdtdVars(FVTypes.EpsIsoBox, %s, var=self.eps)"%eps_args)

if n_dim==3:
    fdtd.SendStrMessage("self.SetViewPlane_Side(%s, [{'A':self.E, 'A_dir':'z', 'O_dir':'x', 'name':'E'}])"%(r_j_str))
elif n_dim<=2:
    print('setting views:')
    fdtd.SendStrMessage("self.SetViewPlane_Side(%s, [{'A':self.E, 'A_dir':'z', 'O_dir':None, 'name':'E'}])"%(r_j_str))


fdtd.SendStrMessage("self.StepFields(%s)"%(n_t))
    
toc = time.perf_counter()
print('\n processing time: {}:{}'.format(int((toc-tic)/60), int((toc-tic)%60)), '\n')


##TODO: resolve corners
##TODO: resolve race conditions


self.pr_grid: [8 1] self.N_chunk: [125 100]
starting process:  [0 0]
starting process:  [1 0]
starting process:  [2 0]
starting process:  [3 0]
self.ChunkInd: [0 0] self.ChunkIndTot: [8 1]
self.ChunkIndNB: {'p': [array([1, 0]), None], 'n': [None, None]}
self.N: [125 100] self.dr: [ 0.01  0.02] self.r0: [ 0.  0.] self.r1: [ 1.25  2.  ]
self.cmdPipe: <multiprocessing.connection.Connection object at 0x7fdc825ac7b8>
self.commPipesNBDicSend:  {(1, 0): <multiprocessing.connection.Connection object at 0x7fdc825ac048>}
self.commPipesNBDicRecv:  {(1, 0): <multiprocessing.connection.Connection object at 0x7fdc825ac4e0>}
self.quitLoop:  False
self.ChunkInd: [1 0] self.ChunkIndTot: [8 1]
self.ChunkIndNB: {'p': [array([2, 0]), None], 'n': [array([0, 0]), None]}
self.N: [125 100] self.dr: [ 0.01  0.02] self.r0: [ 1.25  0.  ] self.r1: [ 2.5  2. ]
self.cmdPipe: <multiprocessing.connection.Connection object at 0x7fdc825ac828>
self.commPipesNBDicSend:  {(2, 0): <multiprocessing.connection.Connection obj

In [8]:
import matplotlib.pyplot as plt

#outs = fdtd.GetOutputs('E')

outs = fdtd.SendStrMessage('A = self.GetOutputs("E")', 'A')
print(outs.keys())

if n_dim==3:
    for i in range(pr_grid[0]):
        for j in range(pr_grid[1]):
            for k in range(pr_grid[2]):
                r_E, E_out_list = outs[(i,j,k)]
                x_E, y_E, z_E = r_E
                e_max = [np.max(np.abs(e)) for e in E_out_list]
                plt.imshow(E_out_list[-1].T)
    plt.show()

if n_dim==2:
    for i in range(pr_grid[0]):
        for j in range(pr_grid[1]):
            r_E, E_out_list = outs[(i,j)]
            x_E, y_E, z_E = [None]*3
            x_E, y_E = r_E
            e_max = max([np.max(np.abs(e)) for e in E_out_list])
            print('e_max:', e_max)
            #plt.imshow(E_out_list[-1].T)
            plt.pcolor(x_E, y_E, E_out_list[-1], vmin=-1.16e-3, vmax=1.16e-3)
    plt.colorbar()
    plt.show()

if n_dim==1:
    for i in range(pr_grid[0]):
        r_E, E_out_list = outs[(i,)]
        x_E = r_E[0]
        e_max = [np.max(np.abs(e)) for e in E_out_list]
        plt.plot(x_E, E_out_list[-1])
        #E_last.append([x_E, E_out_list[-1]])
    plt.show()




dict_keys([(5,), (0,), (6,), (1,), (7,), (2,), (3,), (4,)])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])
Saved outputs: dict_keys(['E'])


In [5]:

fdtd.SendQuitMessage()
time.sleep(0.1)
fdtd.logProcContents()
fdtd.TerminateProcesses()

self.ChunkInd: [0 0 0] self.ChunkIndTot: [1 1 1]
self.ChunkIndNB: {'p': [None, None, None], 'n': [None, None, None]}
self.N: [50 50 50] self.dr: [ 0.16  0.02  0.02] self.r0: [ 0.  0.  0.] self.r1: [ 8.  1.  1.]
self.cmdPipe: <multiprocessing.connection.Connection object at 0x7efd83c61e80>
self.commPipesNBDicSend:  {}
self.commPipesNBDicRecv:  {}
self.quitLoop:  False
